Remove Noisy images

In [ ]:
img_exts = ['jpg', 'jpeg', 'png', 'bmp']

In [ ]:

import tensorflow as tf
import os
import numpy as np
from matplotlib import pyplot as plt
import cv2
import imghdr
from tensorflow.keras.preprocessing import image_dataset_from_directory


data_dir = 'skinType'

for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in img_exts:
                print(image_path)    # print the path of the image with unknown extension
                os.remove(image_path)
        except Exception as e:
            print("Issue with image:" .format(image_path))
            
            


Load Dataset

In [ ]:
batch_size = [16,32]
for size in batch_size:
   data = tf.keras.utils.image_dataset_from_directory('skinType', batch_size=size, image_size=(256, 256), shuffle=True) #Data pipeline

class_names = data.class_names #get the class names
print(class_names)

data_iterator = data.as_numpy_iterator() #allows us to access Data pipeline

batch = data_iterator.next() #get the next batch of data



In [ ]:
batch[0].shape # shape of the batch

In [ ]:
batch[1] # labels

Plotting Images

In [ ]:
from matplotlib import pyplot as plt
rows = 4
cols = 8
fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(20, 12))
fig.suptitle('Sample Images with Class Labels', fontsize=16)

# Flatten axes for easier indexing
axes = axes.flatten()

# Display images in grid
for idx, (image, label) in enumerate(zip(batch[0][:32], batch[1][:32])):
    # Get class name from label index
    class_name = class_names[label]
    
    # Display image
    axes[idx].imshow(image.astype("uint8"))
    axes[idx].set_title(f'{class_name}', fontsize=8)
    axes[idx].axis('off')

# Add color-coded legend
legend_elements = [plt.Line2D([0], [0], marker='o', color='w', 
                             markerfacecolor=f'C{i}', markersize=10, 
                             label=name) for i, name in enumerate(class_names)]
fig.legend(handles=legend_elements, loc='center right')

# Adjust layout
plt.tight_layout()
plt.subplots_adjust(right=0.85)  # Make room for legend
plt.show()

Preprocessing 

Scaling Data

In [ ]:
data = data.map(lambda x,y : ((x/255),y)) #normalizing the data  
scaled_iterator = data.as_numpy_iterator() #allows us to access Data pipeline
batch = scaled_iterator.next() #get the next batch of data

batch[0].max() #max value in the batch

Split data

In [ ]:
len(data)

In [ ]:
train_size = int(len(data) * 0.7) #70% of the data for training
val_size = int(len(data) * 0.2) #20% of the data for validation
test_size = int(len(data) * 0.1) #10% of the data for testing

In [ ]:
print(train_size, val_size, test_size)

In [ ]:
train = data.take(train_size) #take the first 70% of the data for training
val = data.skip(train_size).take(val_size) #skip the first 70% and take the next 20% for validation
test = data.skip(train_size + val_size).take(test_size) #skip the first 90% and take the next 10% for testing

print(len(train), len(val), len(test))

Building Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom, RandomTranslation, RandomContrast, RandomCrop
# filter size = 3x3
# input shape = 256x256x3
# stride = 1
IMAGE_SIZE = 256
CHANNELS = 3
BATCH_SIZE = batch[0].shape[0]
model = Sequential() #initialize the model



data_augmentation = tf.keras.Sequential([
    RandomFlip("horizontal_and_vertical"),         # Flip images both horizontally and vertically
    RandomRotation(0.4),                           # Rotate images up to 40% in both directions
    RandomZoom(height_factor=(-0.2, 0.2),          # Random zoom in/out
               width_factor=(-0.2, 0.2)),
    RandomTranslation(height_factor=0.2,           # Translate images up to 20% in height
                      width_factor=0.2),           # Translate images up to 20% in width
    RandomContrast(0.2),                           # Adjust contrast randomly
    RandomCrop(IMAGE_SIZE - 20, IMAGE_SIZE - 20),  # Crop random parts of the image
    tf.keras.layers.Resizing(IMAGE_SIZE, IMAGE_SIZE)  # Resize back to target size
])


In [ ]:
# Checking what is the expected dimension order for channel
from tensorflow.keras import backend as k
input_shape = (IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
batch_input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
chanDim = -1
if k.image_data_format() == "channels_first":
    input_shape = (CHANNELS, IMAGE_SIZE, IMAGE_SIZE)
    batch_input_shape = (BATCH_SIZE, CHANNELS, IMAGE_SIZE, IMAGE_SIZE)
    chanDim = 1

Add Layers

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Train

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
model_dir = '../saved_models'
log_dir = os.path.join(model_dir, 'logs')
os.makedirs(model_dir, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)

# Define callbacks
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    ModelCheckpoint(
        filepath=os.path.join(model_dir, 'best_model.keras'),
        save_best_only=True,
        monitor='val_accuracy',
        verbose=1
    ),
    TensorBoard(
        log_dir=log_dir,
        histogram_freq=1,
        write_graph=True
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=3,
        min_lr=1e-6,
        verbose=1
    )
]

# Train model
history = model.fit(
    train,
    validation_data=val,
    epochs=50,
    callbacks=callbacks,
    verbose=1
)

# Save final model
final_model_path = os.path.join(model_dir, 'final_model.keras')
model.save(final_model_path)
print(f"Model saved to {final_model_path}")
from matplotlib import pyplot as plt
rows = 4
cols = 8
fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(20, 12))
fig.suptitle('Sample Images with Class Labels', fontsize=16)

# Flatten axes for easier indexing
axes = axes.flatten()

# Display images in grid
for idx, (image, label) in enumerate(zip(batch[0][:32], batch[1][:32])):
    # Get class name from label index
    class_name = class_names[label]
    
    # Display image
    axes[idx].imshow(image.astype("uint8"))
    axes[idx].set_title(f'{class_name}', fontsize=8)
    axes[idx].axis('off')

# Add color-coded legend
legend_elements = [plt.Line2D([0], [0], marker='o', color='w', 
                             markerfacecolor=f'C{i}', markersize=10, 
                             label=name) for i, name in enumerate(class_names)]
fig.legend(handles=legend_elements, loc='center right')

# Adjust layout
plt.tight_layout()
plt.subplots_adjust(right=0.85)  # Make room for legend
plt.show()

print(train_size, val_size, test_size)

train = data.take(train_size) #take the first 70% of the data for training
val = data.skip(train_size).take(val_size) #skip the first 70% and take the next 20% for validation
test = data.skip(train_size + val_size).take(test_size) #skip the first 90% and take the next 10% for testing

print(len(train), len(val), len(test))

In [ ]:
history.history

Overall Code

In [ ]:

import tensorflow as tf
import os
import numpy as np
from matplotlib import pyplot as plt
import cv2
import imghdr
from tensorflow.keras.preprocessing import image_dataset_from_directory

img_exts = ['jpg', 'jpeg', 'png', 'bmp']
data_dir = 'skinType'

for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in img_exts:
                print(image_path)    # print the path of the image with unknown extension
                os.remove(image_path)
        except Exception as e:
            print("Issue with image:" .format(image_path))
batch_size = [16,32]
for size in batch_size:
   data = tf.keras.utils.image_dataset_from_directory('skinType', batch_size=size, image_size=(256, 256), shuffle=True) #Data pipeline

class_names = data.class_names #get the class names
print(class_names)

data_iterator = data.as_numpy_iterator() #allows us to access Data pipeline

batch = data_iterator.next() #get the next batch of data

batch[0].shape # shape of the batch
       
batch[1] # labels

data = data.map(lambda x,y : ((x/255),y)) #normalizing the data  
scaled_iterator = data.as_numpy_iterator() #allows us to access Data pipeline
batch = scaled_iterator.next() #get the next batch of data

batch[0].max() #max value in the batch

len(data)

train_size = int(len(data) * 0.7) #70% of the data for training
val_size = int(len(data) * 0.2) #20% of the data for validation
test_size = int(len(data) * 0.1) #10% of the data for testing


GPT code

In [ ]:
# import os
# import torch
# import torchvision.transforms as transforms
# from torch.utils.data import DataLoader, Dataset
# from transformers import ViTForImageClassification, ViTConfig
# from PIL import Image
# import torch.nn as nn
# import torch.optim as optim

# # Ensure GPU is used if available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# torch.backends.cudnn.benchmark = True

# # Dataset Directories (use raw string to avoid escape sequence issues)
# data_dir = r'D:\FAST\Semester 8\Final Year Project\Skin Disease Dataset\Oily-Dry-Skin-Types'

# # Custom Dataset Class
# class SkinTypeDataset(Dataset):
#     def __init__(self, root_dir, transform=None):
#         self.root_dir = root_dir
#         self.transform = transform
#         self.classes = ['dry', 'normal', 'oily']
#         self.image_paths = []

#         for label in self.classes:
#             class_dir = os.path.join(root_dir, label)
#             if os.path.exists(class_dir):
#                 for img_file in os.listdir(class_dir):
#                     img_path = os.path.join(class_dir, img_file)
#                     if os.path.isfile(img_path):
#                         self.image_paths.append((img_path, label))

#     def __len__(self):
#         return len(self.image_paths)

#     def __getitem__(self, idx):
#         img_path, label = self.image_paths[idx]
#         image = Image.open(img_path).convert('RGB')
#         if self.transform:
#             image = self.transform(image)
#         label = self.classes.index(label)
#         return image, label

# # Data Preprocessing
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
# ])

# # Load Datasets
# train_dataset = SkinTypeDataset(os.path.join(data_dir, 'train'), transform=transform)
# val_dataset = SkinTypeDataset(os.path.join(data_dir, 'valid'), transform=transform)

# train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# # GAN Generator Model
# class Generator(nn.Module):
#     def __init__(self, z_dim, img_channels):
#         super(Generator, self).__init__()
#         self.net = nn.Sequential(
#             nn.Linear(z_dim, 256),
#             nn.ReLU(),
#             nn.Linear(256, 512),
#             nn.ReLU(),
#             nn.Linear(512, 224 * 224 * img_channels),
#             nn.Tanh()
#         )

#     def forward(self, z):
#         output = self.net(z).view(-1, img_channels, 224, 224)
#         return output

# img_channels = 3
# z_dim = 100
# generator = Generator(z_dim, img_channels).to(device)

# # Load Pre-trained ViT Model (with correct classifier size)
# config = ViTConfig.from_pretrained('google/vit-base-patch16-224', num_labels=3)
# model = ViTForImageClassification(config)
# model.to(device)

# # Loss and Optimizers
# criterion = torch.nn.CrossEntropyLoss()
# optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
# gan_optimizer = torch.optim.Adam(generator.parameters(), lr=2e-4)

# # Training Loop
# def train_model(model, train_loader, val_loader, generator, epochs=1):
#     print(f"Training on {device}...")

#     scaler = torch.amp.GradScaler(enabled=(device.type == 'cuda'))  # Handle GPU/CPU
#     for epoch in range(epochs):
#         model.train()
#         generator.train()
#         total_loss = 0

#         for batch_idx, (images, labels) in enumerate(train_loader):
#             images, labels = images.to(device), labels.to(device)

#             # GAN augmentation
#             z = torch.randn(images.size(0), z_dim, device=device)
#             fake_images = generator(z)

#             # Ensure generated images match dimensions
#             if fake_images.shape != images.shape:
#                 fake_images = torch.nn.functional.interpolate(fake_images, size=(224, 224))

#             # Combine real and fake images
#             combined_images = torch.cat((images, fake_images), dim=0)
#             combined_labels = torch.cat((labels, labels), dim=0)

#             # Train ViT Model
#             optimizer.zero_grad()
#             with torch.autocast(device_type=device.type, enabled=(device.type == 'cuda')):
#                 outputs = model(combined_images).logits
#                 loss = criterion(outputs, combined_labels)

#             scaler.scale(loss).backward()
#             scaler.step(optimizer)
#             scaler.update()

#             total_loss += loss.item()

#             # Display Progress
#             if (batch_idx + 1) % 10 == 0:
#                 print(f"Epoch [{epoch + 1}/{epochs}], Step [{batch_idx + 1}/{len(train_loader)}], Loss: {loss.item():.4f}")

#         # Evaluate after each epoch
#         val_acc = evaluate_model(model, val_loader)
#         print(f"Epoch [{epoch + 1}/{epochs}] - Avg Loss: {total_loss / len(train_loader):.4f} - Val Accuracy: {val_acc:.2f}%")

#     torch.save(model.state_dict(), 'skin_type_model.pth')
#     print("Training complete! Model saved as 'skin_type_model.pth'.")

# # Evaluation Function
# def evaluate_model(model, loader):
#     model.eval()
#     correct = 0
#     total = 0

#     with torch.no_grad():
#         for images, labels in loader:
#             images, labels = images.to(device), labels.to(device)
#             outputs = model(images).logits
#             _, predicted = torch.max(outputs, 1)

#             correct += (predicted == labels).sum().item()
#             total += labels.size(0)

#     return 100 * correct / total

# # Train the Model
# train_model(model, train_loader, val_loader, generator)


Trying to make VIT and GAN

In [ ]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import imghdr
from sklearn.metrics import classification_report, roc_auc_score, mean_squared_error
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adagrad, Adadelta
from tensorflow.keras import layers
import json
import torch
# Ensure TensorFlow uses the GPU if available
# GPU Detection using PyTorch
print("Number of GPUs: ", torch.cuda.device_count())
print("GPU Name: ", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU available")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# Dataset directory
data_dir = 'skinType'

# Image Extensions
img_exts = ['jpg', 'jpeg', 'png', 'bmp']

# Data Cleaning
for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in img_exts:
                print("Removing:", image_path)
                os.remove(image_path)
        except Exception as e:
            print("Issue with image:", image_path)

# Parameters
epochs = 3
learning_rates = [0.1, 0.01, 0.001, 0.0001]
batch_sizes = [4, 8, 16, 32, 64]
optimizers = {'adam': Adam, 'sgd': SGD, 'rmsprop': RMSprop, 'adagrad': Adagrad, 'adadelta': Adadelta}

# Store results
results = []

# Load dataset and preprocess
def load_data(batch_size):
    dataset = tf.keras.utils.image_dataset_from_directory(
        data_dir, batch_size=batch_size, image_size=(256, 256), shuffle=True)
    dataset = dataset.map(lambda x, y: (x / 255.0, y))
    return dataset

# Split dataset
def split_data(dataset):
    total_size = len(dataset)
    train_size = int(total_size * 0.7)
    val_size = int(total_size * 0.2)
    test_size = total_size - train_size - val_size

    train_ds = dataset.take(train_size)
    val_ds = dataset.skip(train_size).take(val_size)
    test_ds = dataset.skip(train_size + val_size)

    return train_ds, val_ds, test_ds

# Build simple GAN
def build_gan():
    # Generator
    generator = tf.keras.Sequential([
        layers.Dense(256, activation='relu'),
        layers.Dense(512, activation='relu'),
        layers.Dense(1024, activation='relu'),
        layers.Dense(256 * 256 * 3, activation='tanh'),
        layers.Reshape((256, 256, 3))
    ])

    # Discriminator
    discriminator = tf.keras.Sequential([
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])

    return generator, discriminator

# Build Vision Transformer (ViT)
def build_vit():
    model = tf.keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(len(os.listdir(data_dir)), activation='softmax')
    ])
    return model

# Train and evaluate models
import numpy as np
import torch
import torch.nn.functional as F
from sklearn.metrics import classification_report, roc_auc_score, mean_squared_error
import json

def train_and_evaluate():
    results = []
    
    # Loop over batch sizes
    for batch_size in batch_sizes:
        dataset = load_data(batch_size)
        train_ds, val_ds, test_ds = split_data(dataset)

        # Loop over learning rates
        for lr in learning_rates:
            # Loop over optimizers
            for opt_name, opt_class in optimizers.items():
                optimizer = opt_class(learning_rate=lr)

                # Initialize ViT model
                vit_model = build_vit()
                vit_model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
                
                # Train the ViT model
                history = vit_model.fit(train_ds, validation_data=val_ds, epochs=epochs, verbose=1)

                # Evaluate the ViT model
                y_true = []
                y_pred = []
                y_prob = []  # To store the predicted probabilities

                # Iterate over the test dataset
                for x_batch, y_batch in test_ds:
                    # Move data to GPU if available
                    x_batch = x_batch.to(device)
                    y_batch = y_batch.to(device)

                    # Make predictions using ViT model
                    with torch.no_grad():
                        outputs = vit_model(x_batch)
                        probs = F.softmax(outputs, dim=1)  # Get predicted probabilities
                        _, predicted = torch.max(outputs, 1)  # Get the class predictions

                    y_true.extend(y_batch.cpu().numpy())
                    y_pred.extend(predicted.cpu().numpy())
                    y_prob.extend(probs.cpu().numpy())  # Store the probabilities

                # Compute classification report
                report = classification_report(y_true, y_pred, output_dict=True)

                # Compute ROC AUC score (use probabilities, not hard class predictions)
                auc_score = roc_auc_score(y_true, y_prob, multi_class='ovr', average='macro')

                # Compute RMSE
                rmse = np.sqrt(mean_squared_error(y_true, y_pred))

                # Store the results
                result = {
                    'batch_size': batch_size,
                    'learning_rate': lr,
                    'optimizer': opt_name,
                    'accuracy': report['accuracy'],
                    'precision': report['macro avg']['precision'],
                    'recall': report['macro avg']['recall'],
                    'f1-score': report['macro avg']['f1-score'],
                    'roc_auc': auc_score,
                    'rmse': rmse
                }
                results.append(result)

    # Save the results to a JSON file
    with open('results.json', 'w') as f:
        json.dump(results, f)

    return results

# Example usage (Ensure that the functions `load_data`, `split_data`, `build_vit`, and `optimizers` are defined elsewhere in your code)

# Assuming device is already set up
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# Example batch sizes, learning rates, optimizers, and other parameters
batch_sizes = [32, 64]
learning_rates = [1e-3, 1e-4]
optimizers = {'Adam': torch.optim.Adam, 'SGD': torch.optim.SGD}
epochs = 10

# Call the function
results = train_and_evaluate()

# Optionally, print or plot results
print(results)

# Plot Results
def plot_results(results):
    plt.figure(figsize=(10, 5))
    
    # Accuracy
    accuracies = [res['accuracy'] for res in results]
    plt.plot(accuracies, label='Accuracy')

    # Precision
    precisions = [res['precision'] for res in results]
    plt.plot(precisions, label='Precision')

    # Recall
    recalls = [res['recall'] for res in results]
    plt.plot(recalls, label='Recall')

    # F1-Score
    f1_scores = [res['f1-score'] for res in results]
    plt.plot(f1_scores, label='F1-Score')

    # AUC
    auc_scores = [res['roc_auc'] for res in results]
    plt.plot(auc_scores, label='ROC AUC')

    plt.xlabel("Experiment Index")
    plt.ylabel("Metrics")
    plt.title("Model Performance Metrics")
    plt.legend()
    plt.savefig("model_performance.png")
    plt.show()

# Run training
results = train_and_evaluate()
plot_results(results)


In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import numpy as np

# Step 1: Set up dataset and transformations
data_dir = 'skinType'  # Replace with your dataset path
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to a fixed size
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images (ImageNet mean & std)
])

# Step 2: Load dataset using ImageFolder
dataset = torchvision.datasets.ImageFolder(root=data_dir, transform=transform)

# Step 3: Split dataset into training, validation, and test sets
total_images = len(dataset)
train_size = int(0.7 * total_images)
val_size = int(0.2 * total_images)
test_size = total_images - train_size - val_size
train_indices, test_val_indices = train_test_split(range(total_images), test_size=(val_size + test_size), random_state=42)
val_indices, test_indices = train_test_split(test_val_indices, test_size=(test_size / (val_size + test_size)), random_state=42)

# Create subset datasets
train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)
test_dataset = Subset(dataset, test_indices)

# Step 4: Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Step 5: Define ViT and GAN model (using transformers encoder-decoder)
class ViT(nn.Module):
    def __init__(self, num_classes):
        super(ViT, self).__init__()
        self.vit = torchvision.models.vit_b_16(pretrained=True)  # Using pre-trained ViT model
        self.vit.heads = nn.Linear(self.vit.heads.in_features, num_classes)  # Modify final layer to match number of classes
    
    def forward(self, x):
        return self.vit(x)

class GAN(nn.Module):
    def __init__(self, num_classes):
        super(GAN, self).__init__()
        # Placeholder GAN setup for the sake of illustration
        # In practice, GAN is more complex involving both generator and discriminator
        self.generator = nn.Linear(512, num_classes)
    
    def forward(self, x):
        return self.generator(x)

# Use ViT or GAN based on requirement
model = ViT(num_classes=len(dataset.classes))  # or GAN(num_classes=len(dataset.classes))

# Step 6: Set device and move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Step 7: Define optimizers and loss function
optimizers = [
    torch.optim.Adam(model.parameters(), lr=0.1),
    torch.optim.Adam(model.parameters(), lr=0.01),
    torch.optim.Adam(model.parameters(), lr=0.001),
    torch.optim.Adam(model.parameters(), lr=0.0001)
]

criterion = nn.CrossEntropyLoss()  # Using multiclass CrossEntropyLoss

# Step 8: Training loop
epochs = 3  # Number of epochs

# Store metrics for plotting
all_metrics = {
    'accuracy': [],
    'precision': [],
    'recall': [],
    'f1_score': [],
    'roc_auc': [],
    'error': [],
    'rmse': []
}

for epoch in range(epochs):
    model.train()  # Set the model to training mode
    loop = tqdm(train_loader, leave=True)  # Use tqdm for progress bar

    for step, (images, labels) in enumerate(loop):
        images, labels = images.to(device), labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Get model predictions
        outputs = model(images)

        # Compute the loss
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()  # Update model parameters

        # Get predictions
        _, preds = torch.max(outputs, 1)

        # Collect metrics for evaluation
        acc = accuracy_score(labels.cpu().numpy(), preds.cpu().numpy())
        precision = precision_score(labels.cpu().numpy(), preds.cpu().numpy(), average='weighted')
        recall = recall_score(labels.cpu().numpy(), preds.cpu().numpy(), average='weighted')
        f1 = f1_score(labels.cpu().numpy(), preds.cpu().numpy(), average='weighted')

        # Collect results for plotting and metrics
        all_metrics['accuracy'].append(acc)
        all_metrics['precision'].append(precision)
        all_metrics['recall'].append(recall)
        all_metrics['f1_score'].append(f1)

        # Print loss and metrics every 10 steps
        if step % 10 == 0:
            print(f"Epoch {epoch+1}, Step {step}, Loss: {loss.item()}, Accuracy: {acc}, Precision: {precision}, Recall: {recall}, F1: {f1}")

    # Step 9: Evaluate the model on the validation set
    model.eval()  # Set the model to evaluation mode
    val_labels = []
    val_preds = []

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            val_labels.extend(labels.cpu().numpy())
            val_preds.extend(preds.cpu().numpy())

    # Calculate validation metrics
    val_acc = accuracy_score(val_labels, val_preds)
    val_precision = precision_score(val_labels, val_preds, average='weighted')
    val_recall = recall_score(val_labels, val_preds, average='weighted')
    val_f1 = f1_score(val_labels, val_preds, average='weighted')

    print(f"Validation Accuracy: {val_acc}, Precision: {val_precision}, Recall: {val_recall}, F1: {val_f1}")

# Step 10: Evaluate the model on the test set
test_labels = []
test_preds = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        test_labels.extend(labels.cpu().numpy())
        test_preds.extend(preds.cpu().numpy())

# Calculate test metrics
test_acc = accuracy_score(test_labels, test_preds)
test_precision = precision_score(test_labels, test_preds, average='weighted')
test_recall = recall_score(test_labels, test_preds, average='weighted')
test_f1 = f1_score(test_labels, test_preds, average='weighted')

print(f"Test Accuracy: {test_acc}, Precision: {test_precision}, Recall: {test_recall}, F1: {test_f1}")

# Plot results
plt.figure(figsize=(10, 6))
plt.plot(all_metrics['accuracy'], label='Accuracy')
plt.plot(all_metrics['precision'], label='Precision')
plt.plot(all_metrics['recall'], label='Recall')
plt.plot(all_metrics['f1_score'], label='F1 Score')
plt.xlabel('Step')
plt.ylabel('Metric Value')
plt.legend()
plt.title('Model Performance over Time')
plt.show()

# Save the model and results
torch.save(model.state_dict(), "model.pth")


GAN

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
import numpy as np

# Example Discriminator and Generator Models

class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.fc2 = nn.Linear(256, 512)
        self.fc3 = nn.Linear(512, output_dim)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        return x

class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(input_dim, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 1)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

# Initialize models
input_dim = 100  # Latent space
output_dim = 784  # Flattened 28x28 image (for example)
generator = Generator(input_dim, output_dim)
discriminator = Discriminator(output_dim)

# Loss function and optimizers
criterion = nn.BCELoss()

# Hyperparameters
epochs = 3
batch_sizes = [4, 8, 16, 32, 64]
learning_rates = [0.1, 0.01, 0.001, 0.0001]
optimizers = [optim.Adam(generator.parameters(), lr=lr) for lr in learning_rates]  # example for Adam optimizer

# Data and labels (example)
real_labels = torch.ones(4, 1)  # Batch size 4, for example
fake_labels = torch.zeros(4, 1)

# Random input noise for the generator
z = torch.randn(4, input_dim)

# Training loop
results = []
for epoch in range(epochs):
    print(f"Epoch [{epoch+1}/{epochs}]")
    
    for batch_size in batch_sizes:
        for optimizer in optimizers:
            optimizer.zero_grad()  # Zero gradients before the backward pass
            fake_images = generator(z)
            output = discriminator(fake_images)

            # Calculate Generator loss
            loss_g = criterion(output, real_labels)  # Want discriminator to believe fake images are real
            
            # Backward pass for generator
            torch.autograd.set_detect_anomaly(True)  # Enable anomaly detection
            loss_g.backward()  # Calculate gradients
            optimizer.step()  # Update the generator parameters
            
            # Evaluate the model performance (for this example)
            output_probs = output.detach().numpy()  # Detach to move to numpy for metrics
            
            # Metrics Calculation
            accuracy = accuracy_score(real_labels.numpy(), np.round(output_probs))
            precision = precision_score(real_labels.numpy(), np.round(output_probs))
            recall = recall_score(real_labels.numpy(), np.round(output_probs))
            f1 = f1_score(real_labels.numpy(), np.round(output_probs))
            auc = roc_auc_score(real_labels.numpy(), output_probs)
            
            fpr, tpr, _ = roc_curve(real_labels.numpy(), output_probs)
            plt.plot(fpr, tpr, label=f"LR={optimizer.param_groups[0]['lr']}, Batch={batch_size}")
            
            # Store results
            results.append({
                "epoch": epoch+1,
                "batch_size": batch_size,
                "learning_rate": optimizer.param_groups[0]['lr'],
                "loss": loss_g.item(),
                "accuracy": accuracy,
                "precision": precision,
                "recall": recall,
                "f1_score": f1,
                "auc": auc
            })
            
            # Show results every 10 steps (for simplicity, we will use batch_size as a proxy for steps here)
            if batch_size % 10 == 0:
                print(f"Epoch [{epoch+1}/{epochs}], Batch Size {batch_size}, Loss: {loss_g.item():.4f}, "
                      f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, "
                      f"F1-Score: {f1:.4f}, AUC: {auc:.4f}")
    
    # Save plot for ROC curves after each epoch
    plt.title(f"ROC Curves for Epoch {epoch+1}")
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc='lower right')
    plt.savefig(f"roc_epoch_{epoch+1}.png")
    plt.clf()  # Clear the figure for the next epoch

# Save results to a file
import pandas as pd
df = pd.DataFrame(results)
df.to_csv("training_results.csv", index=False)

# You can also save the models
torch.save(generator.state_dict(), "generator_model.pth")
torch.save(discriminator.state_dict(), "discriminator_model.pth")

print("Training complete! Results saved.")
